In [1]:
# import modules
import shutil
import os
import pandas as pd

In [2]:
'''Defines naming scheme for moved files'''

def rename(file, root, n_sessions, scan_names, destpath):
    
    # use variables from enclosing scope
#     nonlocal n_sessions
#     nonlocal scan_names
#     nonlocal destpath
    
    # viarable to track unsuccessfully renamed files
    problem_file = None
    
    old_path = os.path.join(root, file)
    base, ext = os.path.splitext(file)
    splitpath = root.split('/')

    # get subject ID
    sub = splitpath[-2].split('_')[0]

    # get session number
    ses_number = splitpath[-2].split('_')[1]
    if int(ses_number) <= n_sessions:
        session = 'ses-'+ses_number
    
    else:
        print('unrecognized session number \'' + ses_number + '\' for subID ' + sub)
        problem_file = file
    
    # get scan type (or log)
    if splitpath[-1] == 'ANATOMY':
        runtype = 'anat'
        
    elif splitpath[-1] == 'FUNCTIONAL':
        runtype = 'func'
        
    elif splitpath[-1] == 'LOG':
        runtype = 'LOG'
       
    else:
        print('unrecognized scan or log folder ' + splitpath[-1] + ' for subID ' + sub)
        problem_file = file
        
    # format scan name (or preserve name of log file)
    if base in scan_names:
        new_name = scan_names[base]
        
    elif ext =='log':
        new_name = base
        
    else:
        print('unrecognized scan name ' + base + ' for file ' + file)
        problem_file = file
        
    try:
        if runtype != 'LOG':
            new_path = os.path.join(destpath, sub, session, runtype, sub+'_'+new_name+ext)
        else:
            new_path = os.path.join(destpath, sub, session, runtype, new_name+ext)

    except NameError:
        new_path = None
        
    return new_path, problem_file

In [3]:
'''Copying and renaming files'''

def bidsify(origpath, destpath, n_sessions=2, scan_types=None, scan_names=None, log_changes=True, verbose=True):
    
    # instantiate list of unsuccessfully handled file paths
    problem_files = []
    
    if scan_types is None:
        scan_types = ['anat','func']
        
    if scan_names is None:
        # use default dictionary of scan types
        scan_names = {
            'mprage' : 'T1_w',
            'bold1' : 'task-rest_run-01_bold',
            'bold2' : 'task-mcr_run-02_bold',
            'bold3' : 'task-swm_run-03',
            'bold4' : 'task-dd_run-04',
            'bold5' : 'task-rest_run-05_bold'
        }
        
    if log_changes:
        scan_types.append('log')
        
    
    for i, (root, dirs, files) in enumerate(os.walk(origpath)):
        # create new directory structure
        if i == 0:
            [os.makedirs(destpath+'/'+direc.split('_')[0]+'/ses-'+str(ses+1)+'/'+scantype, exist_ok=True) for direc in dirs for scantype in scan_types for ses in range(n_sessions)]
        
        # move and rename files
        else:
            file_list = [f for f in files if not f.startswith('.')]
            if file_list:
                for file in file_list:
                    old_filepath = os.path.join(root, file)
                    new_filepath, problem_file = rename(file, root, n_sessions, scan_names, destpath)
                    
                    if new_filepath is not None:
                        # move and rename
                        shutil.copy(old_filepath, new_filepath)
                        
                    else:
                        problem_files.append(problem_file)

In [4]:
'''TESTING'''


bidsify('/Users/paxtonfitzpatrick/Desktop/testorig/', '/Users/paxtonfitzpatrick/Desktop/testdest/', log_changes=True)

unrecognized scan name 2018_1_asd-log for file 2018_1_asd-log.txt
unrecognized scan name 2018_1_asd-log for file 2018_1_asd-log.txt
unrecognized scan name 2018_1_asd-log for file 2018_1_asd-log.txt
unrecognized scan name 2018_1_asd-log for file 2018_1_asd-log.txt
unrecognized scan name 2023_1_asd-log for file 2023_1_asd-log.txt
unrecognized scan name example-log for file example-log.txt
unrecognized scan name 2018_1_asd-log for file 2018_1_asd-log.txt
unrecognized scan name 2018_2_jkl-log for file 2018_2_jkl-log.txt
unrecognized scan name 2018_2_jkl-log for file 2018_2_jkl-log.txt
unrecognized scan name 2018_2_jkl-log for file 2018_2_jkl-log.txt
unrecognized scan name 2018_2_jkl-log for file 2018_2_jkl-log.txt


In [21]:
destpath='/Users/paxtonfitzpatrick/Desktop/testdest/'
origpath = '/Users/paxtonfitzpatrick/Desktop/testorig/'
for i, (root, dirs, files) in enumerate(os.walk(origpath)):

    file_list = [f for f in files if not f.startswith('.')]
    if file_list:
        for file in file_list:
            print(file)
            
            old_filepath = os.path.join(root, file)
            print(old_filepath)
            base, ext = os.path.splitext(file)
            print(root)
            print(base)
            print(ext)
# #             base = 'HELLO'
# #             print(base)
#             print('________')

2022_1_asd-anat.txt
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/ANATOMY/2022_1_asd-anat.txt
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/ANATOMY
2022_1_asd-anat
.txt
2022_1_asd-log.txt
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/LOG/2022_1_asd-log.txt
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/LOG
2022_1_asd-log
.txt
example-log.txt
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/LOG/example-log.txt
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/LOG
example-log
.txt
2012_1_asd-func.txt
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/FUNCTIONAL/2012_1_asd-func.txt
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/FUNCTIONAL
2012_1_asd-func
.txt
example-func.txt
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/FUNCTIONAL/example-func.txt
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/FUNCTIONAL
example-func
.txt
2020_1_asd-anat.txt
/Users/paxtonfitzpatrick/Desktop/testorig/2020_1_asd/ANATOMY/2020_1_asd-anat.txt
/Users/paxto

In [43]:
for i, (root, dirs, files) in enumerate(os.walk('/Users/paxtonfitzpatrick/Desktop/testorig/')):
    print(i)
    print(dirs, files)

0
['2020_1_asd', '2019_1_asd', '2021_1_asd', '2018_1_asd', '2018_2_jkl', '2021_2_jkl', '2019_2_jkl', '2020_2_jkl'] ['.DS_Store']
1
['ANATOMICAL', 'LOG', 'FUNCTIONAL'] ['.DS_Store']
2
[] ['.DS_Store', '2020_1_asd-anat.txt']
3
[] ['.DS_Store', '2020_1_asd-log.txt']
4
[] ['.DS_Store', '2020_1_asd-func.txt']
5
['ANATOMICAL', 'LOG', 'FUNCTIONAL'] ['.DS_Store']
6
[] ['.DS_Store', '2019_1_asd-anat.txt']
7
[] ['.DS_Store', '2019_1_asd-log.txt']
8
[] ['2019_1_asd-func.txt', '.DS_Store']
9
['ANATOMICAL', 'LOG', 'FUNCTIONAL'] ['.DS_Store']
10
[] ['.DS_Store', '2021_1_asd-anat.txt']
11
[] ['2018_1_asd-log.txt', 'example-log.txt']
12
[] ['2018_1_asd-func.txt', 'example-func.txt']
13
['ANATOMICAL', 'LOG', 'FUNCTIONAL'] ['.DS_Store']
14
[] ['2018_1_asd-anat.txt', '.DS_Store']
15
[] ['2018_1_asd-log.txt']
16
[] ['2018_1_asd-func.txt']
17
['ANATOMICAL', 'LOG', 'FUNCTIONAL'] ['.DS_Store']
18
[] ['.DS_Store', '2018_2_jkl-anat.txt']
19
[] ['2018_2_jkl-log.txt']
20
[] ['2018_2_jkl-func.txt']
21
['ANATOMICA

In [66]:
origpath = '/Users/paxtonfitzpatrick/Desktop/testorig/'
a, b, = origpath.split('/')[-2], origpath.split('/')[-3]

In [68]:
somefile = '/Users/paxtonfitzpatrick/Desktop/testorig/2018_1_asd/ANATOMICAL/2018_1_asd-anat.txt'
os.path.basename()

'Desktop'

In [74]:
scan_names = {
            'mprage' : 'T1_w',
            'bold1' : 'task-rest_run-01_bold',
            'bold2' : 'task-mcr_run-02_bold',
            'bold3' : 'task-swm_run-03',
            'bold4' : 'task-dd_run-04',
            'bold5' : 'task-rest_run-05_bold'
        }

In [77]:
if 'bold2' in scan_names:
    print(scan_names['bold2'])

task-mcr_run-02_bold


In [1]:
print(asdf)

NameError: name 'asdf' is not defined

In [8]:
def tester():
    hello = None
    print('a')
    return hello

In [11]:
testlist.append(tester())

a


In [12]:
testlist

[None, None]

In [13]:
[i for i in testlist if i is not None]

[]

In [18]:
base, ext = os.path.splitext('Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/ANATOMY/2022_1_asd-anat.txt')

In [23]:
base+''+ext

'example-func.txt'

In [24]:
tt = [i for i in range(20)]

In [41]:
ttt = []

In [42]:
for i in tt:
    i%2==0 and ttt.append(i)

In [43]:
ttt

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]